# Podstawy sztucznej inteligencji
## Ćwiczenie 2 - Wykorzystanie Sztucznych Sieci Neuronowych do rozwiązywania zagadnienia aproksymacji przebiegów w czasie.
### 240645 - Hubert Cywka

## 1. Cel ćwiczenia
Celem pierwszego ćwiczenia jest rozwiązania zagadnienia aproksymacji przebiegów w czasie za pomocą pojedynczego neuronu. Nauka neuronu odbędzie się przy pomocy metody największego spadku.

## 2. Wstęp
Aby zapewnić spójne zachowanie aplikacji przy każdym jej uruchomieniu, ustawiłem stałą wartość ziarna:

In [ ]:
import numpy as np

np.random.seed(42)

Model neuronu można opisać wzorem $y = f(w * x + b)$, gdzie $f$ to liniowa funkcja aktywacji. Dla każdego punktu danych neuron oblicza błąd względem wartości oczekiwanej $y$ stosując wyrażenie $e = y - \hat{y}$, gdzie $\hat{y}$ to wartość wyjściowa obliczona przez neuron, a $y$ to wartość oczekiwana. Podczas procesu uczenia neuron iteracyjnie minimalizuje błąd średniokwadratowy ($MSE$):
 
$$MSE = \frac{1}{n} \sum_{i=1}^{n} \left( y_i - \hat{y}_i \right)^2$$

Korekta wag $w$ i przesunięcia $b$ odbywa się zgodnie z następującymi wzorami: $w_k = w_{k-1} + η * e * x$ oraz $b_k = b_{k-1} * η * e$, gdzie $η$ to współczynnik uczenia. Proces uczenia trwa, dopóki zmiana $MSE$ między kolejnymi iteracjami nie spadnie poniżej określonego progu wynoszącego $0.01$.

## 3. Rozwiązanie
Algorytm realizujący uczenie perceptronu zawarty został w całości w klasie `ApproximationPerceptron` zdefiniowanej poniżej:

In [ ]:
class ApproximationPerceptron:
    def __init__(self, learning_rate):
        self.weight = np.random.random() / 2 
        self.bias = np.random.random() / 2
        self.learning_rate = learning_rate

    def activation(self, x):
        return x

    def predict(self, x):
        z = np.dot(x, self.weight) + self.bias
        return self.activation(z)

    def fit(self, x, y):
        previous_mse = float('inf')
        
        while True:
            total_error = 0 

            for i in range(len(x)):
                y_pred = self.predict(x[i])
                error = y[i] - y_pred

                self.weight += self.learning_rate * error * x[i]
                self.bias += self.learning_rate * error
                total_error += error ** 2

            mse = total_error / len(x)
            delta = abs(mse - previous_mse)
            previous_mse = mse

            if delta <= 0.01:
                break

        return self.weight, self.bias


Zdefiniowałem funkcje pomocnicze do generowania danych treningowych oraz danych testowych:

In [ ]:
def generate_points_for_function(f, x_range, n_points, noise):
    x_values = np.random.uniform(x_range[0], x_range[1], n_points)
    y_values = [f(x) + np.random.uniform(-noise, noise) for x in x_values]
    return np.array(x_values), np.array(y_values)

Wartości $x$ zostały wybrane z przedziału $(0, 1)$. Dla każdej wartości x została wyznaczona wartość $y$, taka, że: $y = 3 * x + 2.5 + a$, gdzie $a$ jest losową wartością z przedziału $(-0.25, 0.25)$. Wygenerowanych zostało 1000 punktów do danych treningowych i 100 punktów do danych testowych.

In [ ]:
import matplotlib.pyplot as plt

def plot_approximation(x, y, perceptron):
    plt.scatter(x, y, color='blue', label='Dane rzeczywiste')

    x_vals = np.linspace(min(x), max(x), 100)
    y_vals = perceptron.weight * x_vals + perceptron.bias
    plt.plot(x_vals, y_vals, color='red', label='Linia aproksymacji')

    plt.title("Aproksymacja z użyciem perceptronu")
    plt.xlabel("x")
    plt.ylabel("y")
    plt.legend()
    plt.grid()
    plt.show()
    
def calculate_relative_error(y_true, y_pred):
    relative_errors = np.abs((y_true - y_pred) / y_true)
    mean_relative_error = np.mean(relative_errors)
    return mean_relative_error

func = lambda x: 3 * x + 2.5
x_range = (0, 1)
noise = 0.25

train_data_x, train_data_y = generate_points_for_function(func, x_range, 1000, noise)
test_data_x, test_data_y = generate_points_for_function(func, x_range, 100, noise)

perceptron = ApproximationPerceptron(0.01)
perceptron.fit(train_data_x, train_data_y)

plot_approximation(test_data_x, test_data_y, perceptron)
test_y_pred = [perceptron.predict(x) for x in train_data_x]
relative_error = calculate_relative_error(np.array(train_data_y), np.array(test_y_pred))
print("Średni błąd względny:", relative_error)

## 4. Wnioski
Ćwiczenie zostało zrealizowane pomyślnie, perceptron poprawnie zaproksymował zdefinowaną wcześniej funkcję $y = 3 * x + 2.5$. Już po trzeciej iteracji nauki udało się osiągnąć średni błąd względny na poziomie 3.49%. 